# Thu thập và phân tích dữ liệu từ một page Facebook
# Bước 1: Thu thập dữ liệu 

# Sử dụng Facebook Scraper để thu thập

In [ ]:
%pip install facebook_scraper pandas numpy
from facebook_scraper import get_posts
import pandas as pd
import numpy as np
import os
import json

In [11]:
FANPAGE_LINK ="Steam"
FOLDER_PATH = "Data/"
COOKIE_PATH = "./mbasic.facebook.com_cookies.txt"

PAGES_NUMBER = 100 # Number of pages to crawl

post_list = []
for post in get_posts(FANPAGE_LINK,
                    options={"comments": True, "reactions": True, "allow_extra_requests": True},
                    extra_info=True, pages=PAGES_NUMBER, cookies=COOKIE_PATH):
    print(post)
    post_list.append(post)

# Chuyển dict này thành Dataframe

In [ ]:
# Initialize dataframe to scrape Facebook post
post_df_full = pd.DataFrame(columns=post_list[0].keys(), index=range(len(post_list)), data=post_list)

# To df
path=FOLDER_PATH + "RawData.csv"
post_df_full.to_csv(path, index=False)
print(path)

In [120]:
# Giờ ta sẽ sử dụng Dataframe đã tạo Valve.csv để phân tích
df = pd.read_csv('Data/RawData.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               598 non-null    int64 
 1   name             598 non-null    object
 2   id_post          598 non-null    object
 3   created_time     598 non-null    object
 4   updated_time     598 non-null    object
 5   message          598 non-null    object
 6   type             598 non-null    object
 7   picture          359 non-null    object
 8   source           54 non-null     object
 9   properties       54 non-null     object
 10  like             598 non-null    int64 
 11  love             598 non-null    int64 
 12  haha             598 non-null    int64 
 13  wow              598 non-null    int64 
 14  sad              598 non-null    int64 
 15  angry            598 non-null    int64 
 16  comments         598 non-null    object
 17  total_reactions  598 non-null    in

# Bước 2: Làm sạch và tiền xử lý dữ liệu

In [121]:
#Drop những column không cần dùng và những column có quá nhiều giá trị NaN
df = df.dropna(thresh=10, axis='columns')
df.columns



Index(['id', 'name', 'id_post', 'created_time', 'updated_time', 'message',
       'type', 'picture', 'source', 'properties', 'like', 'love', 'haha',
       'wow', 'sad', 'angry', 'comments', 'total_reactions', 'total_shares',
       'total_comments'],
      dtype='object')

In [122]:
#Những cột này không có giá trị sử dụng khi phân tích dữ liệu
df = df.drop(columns=['id_post','source','updated_time','properties','id'])

In [123]:
df.head()

,name,created_time,message,type,picture,like,love,haha,wow,sad,angry,comments,total_reactions,total_shares,total_comments
0,Biết thế éo đi làm,2023-11-18T04:13:09+0000,Một ngày làm việc hiệu quả của người lương 5 t...,photo,https://scontent.fhan3-5.fna.fbcdn.net/v/t39.3...,293,3,204,0,6,0,"[{'created_time': '2023-11-18T04:13:24+0000', ...",506,3,19
1,Biết thế éo đi làm,2023-11-18T02:16:00+0000,Khách vote 1 ⭐ vì nhắn lúc 3h sáng không nhân ...,status,NaN,310,0,434,1,18,0,"[{'created_time': '2023-11-18T02:17:04+0000', ...",763,9,47
2,Biết thế éo đi làm,2023-11-17T13:39:28+0000,"""Em yên tâm, chỉ có chị với bé Linh biết chuyệ...",status,NaN,1023,3,1128,0,14,0,"[{'created_time': '2023-11-17T13:41:25+0000', ...",2168,19,110
3,Biết thế éo đi làm,2023-11-17T12:15:30+0000,"Người sếp khi nghe ứng viên nói: ""Lương không ...",photo,https://scontent.fhan3-2.fna.fbcdn.net/v/t39.3...,685,4,485,2,6,0,"[{'created_time': '2023-11-17T12:16:06+0000', ...",1182,8,46
4,Biết thế éo đi làm,2023-11-17T11:14:43+0000,Thế giới đang chia ra thành 2 loại người: Mai ...,status,NaN,755,2,310,3,105,1,"[{'created_time': '2023-11-17T16:23:12+0000', ...",1176,37,147


In [124]:
#Tìm những data bị NaN và xử lý 
df.isna().sum()

name                 0
created_time         0
message              0
type                 0
picture            239
like                 0
love                 0
haha                 0
wow                  0
sad                  0
angry                0
comments             0
total_reactions      0
total_shares         0
total_comments       0
dtype: int64

In [125]:
#Có thể thấy ở cột picture những giá trị NaN có nghĩa là post không có ảnh
df['picture'] = df['picture'].fillna(" ")

In [126]:
#Loại bỏ những giá trị bị duplicate (nếu có)
df.drop_duplicates()

,name,created_time,message,type,picture,like,love,haha,wow,sad,angry,comments,total_reactions,total_shares,total_comments
0,Biết thế éo đi làm,2023-11-18T04:13:09+0000,Một ngày làm việc hiệu quả của người lương 5 t...,photo,https://scontent.fhan3-5.fna.fbcdn.net/v/t39.3...,293,3,204,0,6,0,"[{'created_time': '2023-11-18T04:13:24+0000', ...",506,3,19
1,Biết thế éo đi làm,2023-11-18T02:16:00+0000,Khách vote 1 ⭐ vì nhắn lúc 3h sáng không nhân ...,status,,310,0,434,1,18,0,"[{'created_time': '2023-11-18T02:17:04+0000', ...",763,9,47
2,Biết thế éo đi làm,2023-11-17T13:39:28+0000,"""Em yên tâm, chỉ có chị với bé Linh biết chuyệ...",status,,1023,3,1128,0,14,0,"[{'created_time': '2023-11-17T13:41:25+0000', ...",2168,19,110
3,Biết thế éo đi làm,2023-11-17T12:15:30+0000,"Người sếp khi nghe ứng viên nói: ""Lương không ...",photo,https://scontent.fhan3-2.fna.fbcdn.net/v/t39.3...,685,4,485,2,6,0,"[{'created_time': '2023-11-17T12:16:06+0000', ...",1182,8,46
4,Biết thế éo đi làm,2023-11-17T11:14:43+0000,Thế giới đang chia ra thành 2 loại người: Mai ...,status,,755,2,310,3,105,1,"[{'created_time': '2023-11-17T16:23:12+0000', ...",1176,37,147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,Biết thế éo đi làm,2023-08-22T03:02:15+0000,"""Gặp nhau rồi thật khó l.ìa xa... bao tâm sự c...",photo,https://scontent.fhan3-3.fna.fbcdn.net/v/t39.3...,2699,19,3826,2,59,0,"[{'created_time': '2023-08-22T03:04:06+0000', ...",6605,130,465
594,Biết thế éo đi làm,2023-08-22T01:31:34+0000,Lương 5tr thì ăn bao nhiêu đậu đỏ cũng không c...,status,,2952,13,3009,4,526,1,"[{'created_time': '2023-08-22T01:32:12+0000', ...",6505,282,999
595,Biết thế éo đi làm,2023-08-21T13:05:19+0000,Người zô tri đi phỏng vấn kiểu: \n#BTEDL #agen...,video,https://scontent.fhan4-1.fna.fbcdn.net/v/t15.5...,1201,9,824,0,10,0,"[{'created_time': '2023-09-18T08:03:33+0000', ...",2044,15,12
596,Biết thế éo đi làm,2023-08-21T11:30:30+0000,Tôi sẽ chỉ lấy chồng sau khi đã tìm được công ...,photo,https://scontent.fhan3-4.fna.fbcdn.net/v/t39.3...,3575,77,4716,6,250,1,"[{'created_time': '2023-08-21T18:37:49+0000', ...",8625,751,1083


In [127]:
df['like'][0]

293

In [128]:
df.head(5)


,name,created_time,message,type,picture,like,love,haha,wow,sad,angry,comments,total_reactions,total_shares,total_comments
0,Biết thế éo đi làm,2023-11-18T04:13:09+0000,Một ngày làm việc hiệu quả của người lương 5 t...,photo,https://scontent.fhan3-5.fna.fbcdn.net/v/t39.3...,293,3,204,0,6,0,"[{'created_time': '2023-11-18T04:13:24+0000', ...",506,3,19
1,Biết thế éo đi làm,2023-11-18T02:16:00+0000,Khách vote 1 ⭐ vì nhắn lúc 3h sáng không nhân ...,status,,310,0,434,1,18,0,"[{'created_time': '2023-11-18T02:17:04+0000', ...",763,9,47
2,Biết thế éo đi làm,2023-11-17T13:39:28+0000,"""Em yên tâm, chỉ có chị với bé Linh biết chuyệ...",status,,1023,3,1128,0,14,0,"[{'created_time': '2023-11-17T13:41:25+0000', ...",2168,19,110
3,Biết thế éo đi làm,2023-11-17T12:15:30+0000,"Người sếp khi nghe ứng viên nói: ""Lương không ...",photo,https://scontent.fhan3-2.fna.fbcdn.net/v/t39.3...,685,4,485,2,6,0,"[{'created_time': '2023-11-17T12:16:06+0000', ...",1182,8,46
4,Biết thế éo đi làm,2023-11-17T11:14:43+0000,Thế giới đang chia ra thành 2 loại người: Mai ...,status,,755,2,310,3,105,1,"[{'created_time': '2023-11-17T16:23:12+0000', ...",1176,37,147


In [130]:
#Chuyển Data đã xử lý vào một file csv khác
df.to_csv('Data/Data_Final.csv', index=False)